In [71]:
import json
import re
from neo4j.v1 import GraphDatabase, basic_auth

In [72]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth = basic_auth("neo4j", "temppassword1"))
session = driver.session()

In [73]:
cia = []
with open("../factbook.json") as infile:
    for line in infile:
        cia_fact = json.loads(line)
        cia.append(cia_fact)

In [81]:
for country in cia: #this will go through cia and plot everything
    b = country['region']
    if b == None:
        c = "None"
    else:
        b = b.replace(" ", "")
        c = re.sub('[^a-zA-Z0-9 \n\.]', '', b)
    
    name = country['countryname']
    code = country['countrycode']
    session.run("MERGE (a: "+c+" {name: {name}, code: {code}})", {"name": name, "code": code})
    
    try:  
        print (name)
        gdp = country['sections']['Economy'][2]['category_data'].split('(')[0] #i think some entry doesnt have brackets
        session.run("MATCH (a {name: {name}}) SET a.gdp = {gdp}", {"name": name, "gdp": gdp})
    except:
        print(" ")
    try:
        pop = country['sections']['People and Society'][4]['category_data'].split(' ')[0]
        session.run("MATCH (n {name: {name}}) SET n.population = {pop}", {"name": name, "pop": pop})
    except:
        print(" ")
    
    hasGov = 1
    govern
    try:
        govType = country['sections']['Government'][1]['category_data']
        govern = govType
        session.run("MERGE (b: GovType {type: {govType}})", {"govType": govType})
    except:
        hasGov = 0
    try:
        if(hasGov == 1):
            session.run("MATCH (a), (b)"
                       +"WHERE a.name = {name} AND b.type = {govern}"
                       +"MERGE (a)-[r:Goverment]->(b)",
                       {"name": name, "govern": govern})
    except:
        print(" ")
        
#     try:
#         dispute = country['sections']['Transnational Issues'][0]['category_data']
#         if dispute != "none":
#             session.run("MERGE (c: TransnationalIssue {type: InternatDispute})")
#             session.run("MATCH (a), (b)"
#                        +"WHERE a.name = {name} AND b.type = InternatDispute"
#                        +"MERGE (a)-[r:Is_Dispute {specific: {dispute}}]->(b)",
#                        {"name": name, "dispute": dispute})
#     except:
#         print(dispute)

    try:
        lang = country['sections']['People and Society'][2]['category_data']
        lang = "".join(lang)
        lang = lang.split(',')
        for a in lang:
            temLang = a.split(' ')
            tempLang = temLang[0]
            if temLang[0] == '':
                tempLang = temLang[1]
            session.run("MERGE (a: Language {type: {tempLang}})", {"tempLang": tempLang})
            session.run("MATCH (a), (b)"
                       +"WHERE a.name = {name} AND b.type = {tempLang}"
                       +"MERGE (a)-[r:Speaks]->(b)",
                       {"name": name, "tempLang": tempLang})
    except:
        print(" ")

CypherSyntaxError: Variable `n` not defined (line 1, column 30 (offset: 29))
"MATCH (a {name: {name}}) SET n.gdp = {gdp}"
                              ^

In [78]:
print (cia[122])

{'sections': {'Government': [{'section_title': 'Government', 'category_name': 'Country name', 'category_data': [{'sub_category_name': 'conventional long form', 'sub_category_data': 'none'}, {'sub_category_name': 'conventional short form', 'sub_category_data': 'Jan Mayen'}]}, {'section_title': 'Government', 'category_name': 'Dependency status', 'category_data': 'territory of Norway; since August 1994, administered from Oslo through the county governor (fylkesmann) of Nordland; however, authority has been delegated to a station commander of the Norwegian Defense Communication Service'}, {'section_title': 'Government', 'category_name': 'Legal system', 'category_data': 'the laws of Norway, where applicable, apply'}, {'section_title': 'Government', 'category_name': 'Flag description', 'category_data': 'the flag of Norway is used'}], 'Military': [{'section_title': 'Military', 'category_name': 'Military - note', 'category_data': 'defense is the responsibility of Norway'}], 'Economy': [{'secti

In [11]:
print (cia[6]['sections']['Economy'][2]['category_data'].split('(')[0])

$12.8 billion 
